# Tracking Information Flow

In this chapter, we explore in depth how to track information flows in python by origining input strings, and tracking the origin across string operations.

Some material on `eval` exploitation is adapted from the excellent [blog post](https://nedbatchelder.com/blog/201206/eval_really_is_dangerous.html) by Ned Batchelder.

**Prerequisites**

* You should have read the [chapter on coverage](Coverage.ipynb).
* Some knowledge of inheritance in Python is required.

We first setup our infrastructure so that we can make use of previously defined functions.

In [ ]:
import fuzzingbook_utils

In [ ]:
import Fuzzer

In [ ]:
from ExpectError import ExpectError

Say we want to implement a *calculator* service in Python. A rather easy way to do that is to rely on the `eval()` function in Python. However, unrestricted `eval()` can be used by users to execute arbitrary commands. Since we want to restrict our users to using only the *calculator* functionality, and do not want the users to trash our server, we use `eval()` with empty `locals` and `globals` (as recommended [elsewhere](https://www.programiz.com/python-programming/methods/built-in/eval)).

In [ ]:
def my_calculator(my_input):
    result = eval(my_input, {}, {})
    print("The result of %s is %d" % (my_input, result))

It works as expected:

In [ ]:
my_calculator('1+2')

Does it?

In [ ]:
with ExpectError():
    my_calculator('__import__("os").popen("ls").read()')

As you can see from the error, `eval()` completed successfully, with the system command `ls` executing successfully. It is easy enough for the user to see the output if needed.

In [ ]:
my_calculator(
    "1 if __builtins__['print'](__import__('os').popen('pwd').read()) else 0")

The problem is that the Python `__builtins__` is [inserted by default](https://docs.python.org/3/library/functions.html#eval) when one uses `eval()`. We can avoid this by restricting `__builtins__` in `eval` explicitly (again as recommended [elsewhere](http://lybniz2.sourceforge.net/safeeval.html)).

In [ ]:
def my_calculator(my_input):
    result = eval(my_input, {"__builtins__": None}, {})
    print("The result of %s was %d" % (my_input, result))

Does it help?

In [ ]:
with ExpectError():
    my_calculator(
        "1 if __builtins__['print'](__import__('os').popen('pwd').read()) else 0")

But does it actually?

In [ ]:
my_calculator("1 if [x['print'](x['__import__']('os').popen('pwd').read()) for x in ([x for x in (1).__class__.__base__.__subclasses__() if x.__name__ == 'Sized'][0].__len__.__globals__['__builtins__'],)] else 0")

The problem here is that when the user has a way to inject **uninterpreted strings** that can reach a dangerous routine such as  `eval()` or an `exec()`, it makes it possible for them to inject dangerous code. What we need is a way to restrict the ability of uninterpreted input string fragments from reaching dangerous portions of code.

## Tracking String Taints

There are various levels of taint tracking that one can perform. The simplest is to track that a string fragment originated in a specific environment, and has not undergone a taint removal process. For this, we simply need to wrap the original string with an environment identifier (the _taint_) with `tstr`, and produce `tstr` instances on each operation that results in another string fragment.  

The attribute `taint` holds a label identifying the environment this instance was derived.

For capturing information flows we need a new string class. The idea is to use the new tainted string class `tstr` as a wrapper on the original `str` class. However, `str` is an *immutable* class. Hence, it does not call its `__init__()` method after being constructed. This means that any subclasses of `str` also will not get the `__init__()` method called. If we want to get our initialization routine called, we need to [hook into `__new__()`](https://docs.python.org/3/reference/datamodel.html#basic-customization) and return an instance of our own class.  We combine this with our initialization code in `__init__()`.

In [ ]:
class tstr(str):
    def __new__(cls, value, *args, **kw):
        return str.__new__(cls, value)

    def __init__(self, value, taint=None, **kwargs):
        self.taint = taint

In [ ]:
class tstr(tstr):
    def __repr__(self):
        return tstr(str.__repr__(self), taint=self.taint)

In [ ]:
class tstr(tstr):
    def __str__(self):
        return str.__str__(self)

For example, if we wrap `"hello"` in `tstr`, then we should be able to access its taint:

In [ ]:
thello = tstr('hello', taint='LOW')

In [ ]:
thello.taint

In [ ]:
repr(thello).taint

By default, when we wrap a string, it is tainted. Hence we also need a way to `untaint` the string. One way is to simply return a `str` instance as above. However, one may sometimes wish to remove the taint from an existing instance. This is accomplished with `untaint()`. During `untaint()`, we simply set the taint to `None`. This method comes with a pair method `has_taint()` which checks whether a `tstr` instance is currently origined.

In [ ]:
class tstr(tstr):
    def clear_taint(self):
        self.taint = None
        return self

    def has_taint(self):
        return self.taint is not None

### String Operators

To propagate the taint, we have to extend string functions, such as operators.  We can do so in one single big step, overloading all string methods and operators.

When we create a new string from an existing tainted string, we propagate its taint.

In [ ]:
class tstr(tstr):
    def create(self, s):
        # print("New tstr from", repr(s))
        return tstr(s, taint=self.taint)

The `make_str_wrapper()` function creates a wrapper around an existing string method which attaches the taint to the result of the method:

In [ ]:
def make_str_wrapper(fun):
    def proxy(self, *args, **kwargs):
        res = fun(self, *args, **kwargs)
        # print(fun, args, kwargs, "=", repr(res))
        return self.create(res)
    return proxy

We do this for all string methods that return a string:

In [ ]:
for name in ['__format__', '__getitem__', '__add__', '__mul__', '__rmul__', 
             'capitalize', 'casefold', 'center', 'encode',
            'expandtabs', 'format', 'format_map', 'join', 'ljust', 'lower', 'lstrip', 'replace',
            'rjust', 'rstrip', 'strip', 'swapcase', 'title', 'translate', 'upper']:
    fun = getattr(str, name)
    setattr(tstr, name, make_str_wrapper(fun))

The one missing operator is `+` with a regular string on the left side and a tainted string on the right side.  Python supports a `__radd__()` method which is invoked if the associated object is used on the right side of an addition.

In [ ]:
class tstr(tstr):
    def __radd__(self, s):
        # print("__radd__", repr(s))
        return self.create(s + str(self))

With this, we are already done.  Let us create a string `thello` with a taint `LOW`.

In [ ]:
thello = tstr('hello', taint='LOW')

Now, any substring will also be tainted:

In [ ]:
thello[0].taint

In [ ]:
thello[1:3].taint

String additions will return a `tstr` object with the taint:

In [ ]:
(tstr('foo', taint='HIGH') + 'bar').taint

Our `__radd__()` method ensures this also works if the `tstr` occurs on the right side of a string addition:

In [ ]:
('foo' + tstr('bar', taint='HIGH')).taint

In [ ]:
thello += ', world'

In [ ]:
thello.taint

Other operators such as multiplication also work:

In [ ]:
(thello * 5).taint

## Tracking Untrusted Input

So, what can one do with tainted strings?  We reconsider the `my_calculator()` example.  We define a "better" calculator which only accepts strings tainted as `"TRUSTED"`.

In [ ]:
def better_calculator(s):
    assert isinstance(s, tstr), "Need a tainted string"
    assert s.taint == 'TRUSTED', "Need a string with trusted taint"
    return my_calculator(s)

Feeding a string with an "unknown" (i.e., non-existing) trust level will cause `better_calculator()` to fail:

In [ ]:
with ExpectError():
    better_calculator("2 + 2")

Additionally any user input would be originally tagged with `"UNTRUSTED"` as taint.  If we place an untrusted string into our better calculator, it will also fail:

In [ ]:
bad_user_input = tstr('__import__("os").popen("ls").read()', taint='UNTRUSTED')
with ExpectError():
    better_calculator(bad_user_input)

Hence, somewhere along the computation, we have to turn the "untrusted" inputs into "trusted" strings.  This process is called *sanitization*.  A simple sanitization function for our purposes could ensure that the input consists only of few allowed characters (not including letters or quotes); if this is the case, then the input gets a new `"TRUSTED"` taint.  If not, we turn the string into an (untrusted) empty string; other alternatives would be to raise an error or to escape or delete "untrusted" characters.

In [ ]:
import re

In [ ]:
def sanitize(user_input):
    assert isinstance(user_input, tstr)
    if re.match(r'^[-0-9.+*/%() ]*$', user_input):
        return tstr(user_input, taint='TRUSTED')
    else:
        return tstr('', taint='UNTRUSTED')

In [ ]:
good_user_input = tstr("2 + 2", taint='UNTRUSTED')
sanitized_input = sanitize(good_user_input)
sanitized_input

In [ ]:
sanitized_input.taint

In [ ]:
better_calculator(sanitized_input)

Let us now try out our untrusted input:

In [ ]:
sanitized_input = sanitize(bad_user_input)
sanitized_input

In [ ]:
sanitized_input.taint

In [ ]:
with ExpectError():
    better_calculator(sanitized_input)

In a similar fashion, we can prevent SQL and code injections discussed in [the chapter on Web fuzzing](WebFuzzer.ipynb).

## Preventing Privacy Leaks

Using taints, we can also ensure that secret information does not leak out.  We can assign a special taint `"SECRET"` to strings whose information must not leak out:

In [ ]:
secrets = tstr('<Plenty of secret keys>', taint='SECRET')

Accessing any substring of `secrets` will propagate the taint:

In [ ]:
secrets[1:3].taint

Consider the _heartbeat_ security leak from [the chapter on Fuzzing](Fuzzer.ipynb), in which a server would accidentally reply not only the user input sent to it, but also secret memory.  If the reply consists only of the user input, there is no taint associated with it:

In [ ]:
user_input = "hello"
reply = user_input

In [ ]:
isinstance(reply, tstr)

If, however, the reply contains _any_ part of the secret, the reply will be tainted:

In [ ]:
reply = user_input + secrets[0:5]

In [ ]:
reply

In [ ]:
reply.taint

The output function of our server would now ensure that the data sent back does not contain any secret information:

In [ ]:
def send_back(s):
    assert not isinstance(s, tstr) and not s.taint == 'SECRET'
    ...

In [ ]:
with ExpectError():
    send_back(reply)

## Tracking Character Origins

Our `tstr` solution can be help to identify information leaks – but it is by no means complete.  If we actually take the `heartbeat()` implementation from [the chapter on Fuzzing](Fuzzer.ipynb), we will see that _any_ reply is marked as `SECRET` – even those not even accessing secret memory:

In [ ]:
from Fuzzer import heartbeat

In [ ]:
reply = heartbeat('hello', 5, memory=secrets)

In [ ]:
reply.taint

Why is this?  If we look into the implementation of `heartbeat()`, we will see that it first builds a long string `memory` from the (non-secret) reply and the (secret) memory, before returning the first characters from `memory`.
```python
    # Store reply in memory
    memory = reply + memory[len(reply):]
```
At this point, the whole memory still is tainted as `SECRET`, _including_ the non-secret part from `reply`.

We may be able to circumvent the issue by tagging the `reply` as `PUBLIC` – but then, this taint would be in conflict with the `SECRET` tag of `memory`.  What happens if we compose a string from two differently tainted strings?

In [ ]:
thilo = tstr("High", taint='HIGH') + tstr("Low", taint='LOW')

It turns out that in this case, the `__add__()` method takes precedence over the `__radd__()` method, which means that the right-hand `"Low"` string is treated as a regular (non-tainted) string.

In [ ]:
thilo

In [ ]:
thilo.taint

We could set up the `__add__()` and other methods with special handling for conflicting taints.  However, the way this conflict should be resolved would be highly _application-dependent_:

* If we use taints to indicate _privacy levels_, `SECRET` privacy should take precedence over `PUBLIC` privacy.  Any combination of a `SECRET`-tainted string and a `PUBLIC`-tainted string thus should have a `SECRET` taint.

* If we use taints to indicate _origins_ of information, an `UNTRUSTED` origin should take precedence over a `TRUSTED` origin.  Any combination of an `UNTRUSTED`-tainted string and a `TRUSTED`-tainted string thus should have an `UNTRUSTED` taint.

Of course, such conflict resolutions can be implemented.  But even so, they will not help us in the `heartbeat()` example differentiating secret from non-secret output data.

### Tracking Individual Characters

Fortunately, there is a better, more generic way to solve the above problems.  The key to composition of differently tainted strings is to assign taints not only to strings, but actually to every bit of information – in our case, characters.  If every character has a taint on its own, a new composition of characters will simply inherit this very taint _per character_.  To this end, we introduce a second bit of information named _origin_.

Distinguishing various untrusted sources may be accomplished by origining each instances as separate instances (called *colors* in dynamic origin research). You will see an instance of this technique in the chapter on [Grammar Mining](GrammarMiner.ipynb).

In this section, we carry *character level* origins. That is, given a fragment that resulted from a portion of the original origined string, one will be able to tell which portion of the input string the fragment was taken from. In essence, each input character index from a origined source gets its own color.

More complex origining such as *bitmap origins* are possible where a single character may result from multiple origined character indexes (such as *checksum* operations on strings). We do not consider these in this chapter.

### A Class for Tracking Character Origins

Let us introduce a class `ostr` which, like `tstr`, carries a taint for each string, and additionally an _origin_ for each character that indicates its source.  It is a consecutive number in a particular range (by default, starting with zero) indicating its _position_ within a specific origin.

In [ ]:
class ostr(str):
    DEFAULT_ORIGIN = 0

    def __new__(cls, value, *args, **kw):
        return str.__new__(cls, value)

    def __init__(self, value, taint=None, origin=None, **kwargs):
        self.taint = taint

        if origin is None:
            origin = ostr.DEFAULT_ORIGIN
        if isinstance(origin, int):
            self.origin = list(range(origin, origin + len(self)))
        else:
            self.origin = origin
        assert len(self.origin) == len(self)

In [ ]:
class ostr(ostr):
    def create(self, s):
        return ostr(s, taint=self.taint, origin=self.origin)

In [ ]:
class ostr(ostr):
    UNKNOWN_ORIGIN = -1
    def __repr__(self):
        return ostr(str.__repr__(self), taint=self.taint, 
                    origin=[ostr.UNKNOWN_ORIGIN] + self.origin + [ostr.UNKNOWN_ORIGIN])

In [ ]:
class ostr(ostr):
    def __str__(self):
        return str.__str__(self)

By default, character origins start with `0`:

In [ ]:
thello = ostr('hello')
assert thello.origin == [0, 1, 2, 3, 4]

We can also specify the starting origin as below -- `6..10`

In [ ]:
tworld = ostr('world', origin=6)
assert tworld.origin == [6, 7, 8, 9, 10]

`str()` returns an `str` instance without origin or taint information:

In [ ]:
assert type(str(thello)) == str

`repr()`, however, keeps the origin information for the original string:

In [ ]:
repr(thello)

In [ ]:
repr(thello).origin

Just as with taints, we can clear origins and check whether an origin is present:

In [ ]:
class ostr(ostr):
    def clear_taint(self):
        self.taint = None
        return self

    def has_taint(self):
        return self.taint is not None

In [ ]:
class ostr(ostr):
    def clear_origin(self):
        self.origin = [self.UNKNOWN_ORIGIN] * len(self)
        return self

    def has_origin(self):
        return any(origin != self.UNKNOWN_ORIGIN for origin in self.origin)

In [ ]:
thello = ostr('Hello')
assert thello.has_origin()

In [ ]:
thello.clear_origin()
assert not thello.has_origin()

In the remainder of this section, we re-implement various string methods such that they also keep track of origins.  If this is too tedious for you, jump right [to the next section](#Checking-Origins) which gives a number of usage examples.

### Create

We need to create new substrings that are wrapped in `ostr` objects. However, we also want to allow our subclasses to create their own instances. Hence we again provide a `create()` method that produces a new `ostr` instance.

In [ ]:
class ostr(ostr):
    def create(self, res, origin=None):
        return ostr(res, taint=self.taint, origin=origin)

In [ ]:
thello = ostr('hello', taint='HIGH')
tworld = thello.create('world', origin=6)

In [ ]:
tworld.origin

In [ ]:
tworld.taint

In [ ]:
assert (thello.origin, tworld.origin) == (
    [0, 1, 2, 3, 4], [6, 7, 8, 9, 10])

### Index

In Python, indexing is provided through `__getitem__()`. Indexing on positive integers is simple enough. However, it has two additional wrinkles. The first is that, if the index is negative, that many characters are counted from the end of the string which lies just after the last character. That is, the last character has a negative index `-1`

In [ ]:
class ostr(ostr):
    def __getitem__(self, key):
        res = super().__getitem__(key)
        if isinstance(key, int):
            key = len(self) + key if key < 0 else key
            return self.create(res, [self.origin[key]])
        elif isinstance(key, slice):
            return self.create(res, self.origin[key])
        else:
            assert False

In [ ]:
hello = ostr('hello', taint='HIGH')
assert (hello[0], hello[-1]) == ('h', 'o')
hello[0].taint

The other wrinkle is that `__getitem__()` can accept a slice. We discuss this next.

### Slices

The Python `slice` operator `[n:m]` relies on the object being an `iterator`. Hence, we define the `__iter__()` method, which returns a custom `iterator`.

In [ ]:
class ostr(ostr):
    def __iter__(self):
        return ostr_iterator(self)

The `__iter__()` method requires a supporting `iterator` object. The `iterator` is used to save the state of the current iteration, which it does by keeping a reference to the original `ostr`, and the current index of iteration `_str_idx`.

In [ ]:
class ostr_iterator():
    def __init__(self, ostr):
        self._ostr = ostr
        self._str_idx = 0

    def __next__(self):
        if self._str_idx == len(self._ostr):
            raise StopIteration
        # calls ostr getitem should be ostr
        c = self._ostr[self._str_idx]
        assert isinstance(c, ostr)
        self._str_idx += 1
        return c

Bringing all these together:

In [ ]:
thw = ostr('hello world', taint='HIGH')
thw[0:5]

In [ ]:
assert thw[0:5].has_taint()
assert thw[0:5].has_origin()

In [ ]:
thw[0:5].taint

In [ ]:
thw[0:5].origin

### Splits

In [ ]:
def make_split_wrapper(fun):
    def proxy(self, *args, **kwargs):
        lst = fun(self, *args, **kwargs)
        return [self.create(elem) for elem in lst]
    return proxy

In [ ]:
for name in ['split', 'rsplit', 'splitlines']:
    fun = getattr(str, name)
    setattr(ostr, name, make_split_wrapper(fun))

In [ ]:
thello = ostr('hello world', taint='LOW')
thello == 'hello world'

In [ ]:
thello.split()[0].taint

\todo{Partitions!}

### Concatenation

If two origined strings are concatenated together, it may be desirable to transfer the origins from each to the corresponding portion of the resulting string. The concatenation of strings is accomplished by overriding `__add__()`.

In [ ]:
class ostr(ostr):
    def __add__(self, other):
        if isinstance(other, ostr):
            return self.create(str.__add__(self, other),
                               (self.origin + other.origin))
        else:
            return self.create(str.__add__(self, other),
                               (self.origin + [self.UNKNOWN_ORIGIN for i in other]))

In [ ]:
thello = ostr("hello")
tworld = ostr("world", origin=6)
thw = thello + tworld
assert thw.origin == [0, 1, 2, 3, 4, 6, 7, 8, 9, 10]

What if a `ostr` is concatenated with a `str`?

In [ ]:
space = "  "
th_w = thello + space + tworld
assert th_w.origin == [0, 1, 2, 3, 4, ostr.UNKNOWN_ORIGIN, ostr.UNKNOWN_ORIGIN, 6, 7, 8, 9, 10]

One wrinkle here is that when adding a `ostr` and a `str`, the user may place the `str` first, in which case, the `__add__()` method will be called on the `str` instance. Not on the `ostr` instance. However, Python provides a solution. If one defines `__radd__()` on the `ostr` instance, that method will be called rather than `str.__add__()`

In [ ]:
class ostr(ostr):
    def __radd__(self, other):
        origin = other.origin if isinstance(other, ostr) else [
            self.UNKNOWN_ORIGIN for i in other]
        return self.create(str.__add__(other, self), (origin + self.origin))

We test it out:

In [ ]:
shello = "hello"
tworld = ostr("world")
thw = shello + tworld
assert thw.origin == [ostr.UNKNOWN_ORIGIN] * len(shello) + [0, 1, 2, 3, 4]

These methods: `slicing` and `concatenation` is sufficient to implement other string methods that result in a string, and does not change the character underneath (i.e no case change). Hence, we look at a helper method next.

### Extract Origin String

Given a specific input index, the method `x()` extracts the corresponding origined portion from a `ostr`. As a convenience it supports `slices` along with `ints`.

In [ ]:
class ostr(ostr):
    class TaintException(Exception):
        pass

    def x(self, i=0):
        if not self.origin:
            raise origin.TaintException('Invalid request idx')
        if isinstance(i, int):
            return [self[p]
                    for p in [k for k, j in enumerate(self.origin) if j == i]]
        elif isinstance(i, slice):
            r = range(i.start or 0, i.stop or len(self), i.step or 1)
            return [self[p]
                    for p in [k for k, j in enumerate(self.origin) if j in r]]

In [ ]:
thw = ostr('hello world', origin=100)

In [ ]:
assert thw.x(101) == ['e']

In [ ]:
assert thw.x(slice(101, 105)) == ['e', 'l', 'l', 'o']

### Replace

The `replace()` method replaces a portion of the string with another.

In [ ]:
class ostr(ostr):
    def replace(self, a, b, n=None):
        old_origin = self.origin
        b_origin = b.origin if isinstance(b, ostr) else [self.UNKNOWN_ORIGIN] * len(b)
        mystr = str(self)
        i = 0
        while True:
            if n and i >= n:
                break
            idx = mystr.find(a)
            if idx == -1:
                break
            last = idx + len(a)
            mystr = mystr.replace(a, b, 1)
            partA, partB = old_origin[0:idx], old_origin[last:]
            old_origin = partA + b_origin + partB
            i += 1
        return self.create(mystr, old_origin)

In [ ]:
my_str = ostr("aa cde aa")
res = my_str.replace('aa', 'bb')
assert res, res.origin == ('bb', 'cde', 'bb',
                          [self.UNKNOWN_ORIGIN, self.UNKNOWN_ORIGIN, 
                           2, 3, 4, 5, 6,
                           self.UNKNOWN_ORIGIN, self.UNKNOWN_ORIGIN])

In [ ]:
my_str = ostr("aa cde aa")
res = my_str.replace('aa', ostr('bb', origin=100))
assert (res, res.origin) == (('bb cde bb'), [100, 101, 2, 3, 4, 5, 6, 100, 101])

### Split

We essentially have to re-implement split operations, and split by space is slightly different from other splits.

In [ ]:
class ostr(ostr):
    def _split_helper(self, sep, splitted):
        result_list = []
        last_idx = 0
        first_idx = 0
        sep_len = len(sep)

        for s in splitted:
            last_idx = first_idx + len(s)
            item = self[first_idx:last_idx]
            result_list.append(item)
            first_idx = last_idx + sep_len
        return result_list

    def _split_space(self, splitted):
        result_list = []
        last_idx = 0
        first_idx = 0
        sep_len = 0
        for s in splitted:
            last_idx = first_idx + len(s)
            item = self[first_idx:last_idx]
            result_list.append(item)
            v = str(self[last_idx:])
            sep_len = len(v) - len(v.lstrip(' '))
            first_idx = last_idx + sep_len
        return result_list

    def rsplit(self, sep=None, maxsplit=-1):
        splitted = super().rsplit(sep, maxsplit)
        if not sep:
            return self._split_space(splitted)
        return self._split_helper(sep, splitted)

    def split(self, sep=None, maxsplit=-1):
        splitted = super().split(sep, maxsplit)
        if not sep:
            return self._split_space(splitted)
        return self._split_helper(sep, splitted)

In [ ]:
my_str = ostr('ab cdef ghij kl')
ab, cdef, ghij, kl = my_str.rsplit(sep=' ')
assert (ab.origin, cdef.origin, ghij.origin,
        kl.origin) == ([0, 1], [3, 4, 5, 6], [8, 9, 10, 11], [13, 14])

my_str = ostr('ab cdef ghij kl', origin=list(range(0, 15)))
ab, cdef, ghij, kl = my_str.rsplit(sep=' ')
assert(ab.origin, cdef.origin, kl.origin) == ([0, 1], [3, 4, 5, 6], [13, 14])

In [ ]:
my_str = ostr('ab   cdef ghij    kl', origin=100, taint='HIGH')
ab, cdef, ghij, kl = my_str.rsplit()
assert (ab.origin, cdef.origin, ghij.origin,
        kl.origin) == ([100, 101], [105, 106, 107, 108], [110, 111, 112, 113],
                      [118, 119])

my_str = ostr('ab   cdef ghij    kl', origin=list(range(0, 20)), taint='HIGH')
ab, cdef, ghij, kl = my_str.split()
assert (ab.origin, cdef.origin, kl.origin) == ([0, 1], [5, 6, 7, 8], [18, 19])
assert ab.taint == 'HIGH'

### Strip

In [ ]:
class ostr(ostr):
    def strip(self, cl=None):
        return self.lstrip(cl).rstrip(cl)

    def lstrip(self, cl=None):
        res = super().lstrip(cl)
        i = self.find(res)
        return self[i:]

    def rstrip(self, cl=None):
        res = super().rstrip(cl)
        return self[0:len(res)]

In [ ]:
my_str1 = ostr("  abc  ")
v = my_str1.strip()
assert v, v.origin == ('abc', [2, 3, 4])

In [ ]:
my_str1 = ostr("  abc  ")
v = my_str1.lstrip()
assert (v, v.origin) == ('abc  ', [2, 3, 4, 5, 6])

In [ ]:
my_str1 = ostr("  abc  ")
v = my_str1.rstrip()
assert (v, v.origin) == ('  abc', [0, 1, 2, 3, 4])

### Expand Tabs

In [ ]:
class ostr(ostr):
    def expandtabs(self, n=8):
        parts = self.split('\t')
        res = super().expandtabs(n)
        all_parts = []
        for i, p in enumerate(parts):
            all_parts.extend(p.origin)
            if i < len(parts) - 1:
                l = len(all_parts) % n
                all_parts.extend([p.origin[-1]] * l)
        return self.create(res, all_parts)

In [ ]:
my_str = str("ab\tcd")
my_ostr = ostr("ab\tcd")
v1 = my_str.expandtabs(4)
v2 = my_ostr.expandtabs(4)

In [ ]:
assert str(v1) == str(v2)
assert (len(v1), repr(v2), v2.origin) == (6, "'ab  cd'", [0, 1, 1, 1, 3, 4])

In [ ]:
class ostr(ostr):
    def join(self, iterable):
        mystr = ''
        myorigin = []
        sep_origin = self.origin
        lst = list(iterable)
        for i, s in enumerate(lst):
            sorigin = s.origin if isinstance(s, ostr) else [self.UNKNOWN_ORIGIN] * len(s)
            myorigin.extend(sorigin)
            mystr += str(s)
            if i < len(lst) - 1:
                myorigin.extend(sep_origin)
                mystr += str(self)
        res = super().join(iterable)
        assert len(res) == len(mystr)
        return self.create(res, myorigin)

In [ ]:
my_str = ostr("ab cd", origin=100)
(v1, v2), v3 = my_str.split(), 'ef'
assert (v1.origin, v2.origin) == ([100, 101], [103, 104])

In [ ]:
v4 = ostr('').join([v2, v3, v1])
assert (v4, v4.origin) == ('cdefab', [103, 104, ostr.UNKNOWN_ORIGIN, ostr.UNKNOWN_ORIGIN, 100, 101])

In [ ]:
my_str = ostr("ab cd", origin=100)
(v1, v2), v3 = my_str.split(), 'ef'
assert (v1.origin, v2.origin) == ([100, 101], [103, 104])

In [ ]:
v4 = ostr(',').join([v2, v3, v1])
assert (v4, v4.origin) == ('cd,ef,ab', 
                           [103, 104, 0, ostr.UNKNOWN_ORIGIN, ostr.UNKNOWN_ORIGIN, 0, 100, 101])

### Partitions

In [ ]:
class ostr(ostr):
    def partition(self, sep):
        partA, sep, partB = super().partition(sep)
        return (self.create(partA, self.origin[0:len(partA)]),
                self.create(sep, self.origin[len(partA):len(partA) + len(sep)]),
                self.create(partB, self.origin[len(partA) + len(sep):]))

    def rpartition(self, sep):
        partA, sep, partB = super().rpartition(sep)
        return (self.create(partA, self.origin[0:len(partA)]),
                self.create(sep, self.origin[len(partA):len(partA) + len(sep)]),
                self.create(partB, self.origin[len(partA) + len(sep):]))

### Justify

In [ ]:
class ostr(ostr):
    def ljust(self, width, fillchar=' '):
        res = super().ljust(width, fillchar)
        initial = len(res) - len(self)
        if isinstance(fillchar, tstr):
            t = fillchar.x()
        else:
            t = self.UNKNOWN_ORIGIN
        return self.create(res, [t] * initial + self.origin)

In [ ]:
class ostr(ostr):
    def rjust(self, width, fillchar=' '):
        res = super().rjust(width, fillchar)
        final = len(res) - len(self)
        if isinstance(fillchar, tstr):
            t = fillchar.x()
        else:
            t = self.UNKNOWN_ORIGIN
        return self.create(res, self.origin + [t] * final)

### String methods that do not change origin

In [ ]:
class ostr(ostr):
    def swapcase(self):
        return self.create(str(self).swapcase(), self.origin)

    def upper(self):
        return self.create(str(self).upper(), self.origin)

    def lower(self):
        return self.create(str(self).lower(), self.origin)

    def capitalize(self):
        return self.create(str(self).capitalize(), self.origin)

    def title(self):
        return self.create(str(self).title(), self.origin)

In [ ]:
a = ostr('aa', origin=100).upper()
a, a.origin

### General wrappers

These are not strictly needed for operation, but can be useful for tracing.

In [ ]:
def make_str_wrapper(fun):
    def proxy(*args, **kwargs):
        res = fun(*args, **kwargs)
        return res
    return proxy

In [ ]:
import inspect

In [ ]:
import types

In [ ]:
ostr_members = [name for name, fn in inspect.getmembers(ostr, callable)
                if isinstance(fn, types.FunctionType) and fn.__qualname__.startswith('ostr')]

for name, fn in inspect.getmembers(str, callable):
    if name not in set(['__class__', '__new__', '__str__', '__init__',
                        '__repr__', '__getattribute__']) | set(ostr_members):
        setattr(ostr, name, make_str_wrapper(fn))

### Methods yet to be translated

These methods generate strings from other strings. However, we do not have the right implementations for any of these. Hence these are marked as dangerous until we can generate the right translations.

In [ ]:
def make_str_abort_wrapper(fun):
    def proxy(*args, **kwargs):
        raise ostr.TaintException('%s Not implemented in `ostr`' % fun.__name__)
    return proxy

In [ ]:
for name, fn in inspect.getmembers(str, callable):
    # Omitted 'splitlines' as this is needed for formatting output in IPython/Jupyter
    if name in ['__format__', '__rmod__', '__mod__', 'format_map', 'format',
                '__mul__', '__rmul__', 'center', 'zfill', 'decode', 'encode']:
        setattr(ostr, name, make_str_abort_wrapper(fn))

## Checking Origins

With all this implemented, we now have full-fledged `ostr` strings where we can easily check the origin of each and every character.  

To check whether a string originates from another string, we can convert the origin to a set and resort to standard set operations:

In [ ]:
s = ostr("hello", origin=100)
s[1]

In [ ]:
s[1].origin

In [ ]:
set(s[1].origin) <= set(s.origin)

In [ ]:
t = ostr("world", origin=200)

In [ ]:
set(s.origin) <= set(t.origin)

In [ ]:
u = s + t + "!"

In [ ]:
u.origin

In [ ]:
ostr.UNKNOWN_ORIGIN in u.origin

## Privacy Leaks Revisited

Let us apply it to see whether we can come up with a satisfactory solution for checking the `heartbeat()` function against information leakage.

In [ ]:
SECRET_ORIGIN = 1000

We define a "secret" that must not leak out:

In [ ]:
secret = ostr('<again, some super-secret input>', origin=SECRET_ORIGIN)

Each and every character in `secret` has an origin starting with `SECRET_ORIGIN`:

In [ ]:
print(secret.origin)

If we now invoke `heartbeat()` with a given string, the origin of the reply should all be `UNKNOWN_ORIGIN` (from the input), and none of the characters should have a `SECRET_ORIGIN`.

In [ ]:
s = heartbeat('hello', 5, memory=secret)
s

In [ ]:
print(s.origin)

We can verify that the secret did not leak out by formulating appropriate assertions:

In [ ]:
assert s.origin == [ostr.UNKNOWN_ORIGIN] * len(s)

In [ ]:
assert all(origin == ostr.UNKNOWN_ORIGIN for origin in s.origin)

In [ ]:
assert not any(origin >= SECRET_ORIGIN for origin in s.origin)

All assertions pass, again confirming that no secret leaked out.

Let us now go and exploit `heartbeat()` to reveal its secrets.  As `heartbeat()` is unchanged, it is as vulnerable as it was:

In [ ]:
s = heartbeat('hello', 32, memory=secret)
s

Now, however, the reply _does_ contain secret information:

In [ ]:
print(s.origin)

In [ ]:
with ExpectError():
    assert s.origin == [ostr.UNKNOWN_ORIGIN] * len(s)

In [ ]:
with ExpectError():
    assert all(origin == ostr.UNKNOWN_ORIGIN for origin in s.origin)

In [ ]:
with ExpectError():
    assert not any(origin >= SECRET_ORIGIN for origin in s.origin)

We can now integrate these assertions into the `heartbeat()` function, causing it to fail before leaking information.  Additionally (or alternatively?), we can also rewrite our output functions not to give out any secret information.  We will leave these two exercises for the reader.

## The Limits of Information Flow

While our framework can detect information leakage, it is by no means perfect.  There are several ways in which taints can get lost and information thus may still leak out.

### Conversions

We only track taints and origins through _strings_ and _characters_.  If we convert these to numbers (or other data), the information is lost.

As an example, consider this function, converting individual characters to numbers and back:

In [ ]:
def strip_all_info(s):
    t = ""
    for c in s:
        t += chr(ord(c))
    return t

In [ ]:
thello = ostr("Secret")
thello

In [ ]:
thello.origin

The taints and origins will not propagate through the number conversion:

In [ ]:
thello_stripped = strip_all_info(thello)
thello_stripped

In [ ]:
with ExpectError():
    thello_stripped.origin

This issue could be addressed by extending numbers with taints and origins, just as we did for strings.  At some point, however, this will still break down, because as soon as an internal C function in the Python library is reached, the taint will not propagate into and across the C function.  (Unless one starts implementing dynamic taints for these, that is.)

### Implicit Information Flow

Even if one could taint all data in a program, there still would be means to break information flow – notably by turning explicit flow into _implicit_ flow, or data flow into _control flow_.  Here is an example:

In [ ]:
def strip_all_info_again(s):
    t = ""
    for c in s:
        if c == 'a':
            t += 'a'
        elif c == 'b':
            t += 'b'
        elif c == 'c':
            t += 'c'
    ...

With such a function, there is no explicit data flow between the characters in `s` and the characters in `t`; yet, the strings would be identical.  This problem frequently occurs in programs that process and manipulate external input.

### Enforcing Tainting

Both conversions and implicit information flow are one of several possibilities that taint and origin information get lost.  To address the problem, the best solution is to _always assume the worst from untainted strings_:

* As it comes to trust, an untainted string should be treated as _possibly untrusted_, and hence not relied upon unless sanitized.

* As it comes to privacy, an untainted string should be treated as _possibly secret_, and hence not leaked out.

As a consequence, your program should always have two kinds of taints: one for explicitly trusted (or secret) and one for explicitly untrusted (or non-secret).  If a taint gets lost along the way, you will may have to restore it from its sources – not unlike the string methods discussed above.  The benefit is a trusted application, in which each and every information flow can be checked at runtime, with violations quickly discovered through automated tests.

## Lessons Learned

* String-based and character-based taints allow to dynamically track the information flow form input to the internals of a system and back to the output.

* Checking taints allows to discover untrusted inputs and information leakage at runtime.

* Data conversions and implicit data flow may strip taint information; the resulting untainted strings should be treated as having the worst possible taint.

## Next Steps

_Link to subsequent chapters (notebooks) here:_

## Background

\todo{Add.}

\cite{Lin2008}

## Exercises

### Exercise 1: Tainted Numbers

Introduce a class `tint` (for tainted integer) that, like `tstr`, has a taint attribute that gets passed on from `tint` to `tint`.

#### Part 1: Creation

Implement the `tint` class such that taints are set:

```python
x = tint(42, taint='SECRET')
assert x.taint == 'SECRET'
```

**Solution.** This is pretty straightforward, as we can apply the same scheme as for `tstr`:

In [ ]:
class tint(int):
    def __new__(cls, value, *args, **kw):
        return int.__new__(cls, value)

    def __init__(self, value, taint=None, **kwargs):
        self.taint = taint

In [ ]:
x = tint(42, taint='SECRET')
assert x.taint == 'SECRET'

#### Part 2: Arithmetic expressions

Ensure that taints get passed along arithmetic expressions; support addition, subtraction, multiplication, and division operators.
```python
y = x + 1
assert y.taint == 'SECRET'
```

**Solution.** As with `tstr`, we implement a `create()` method and a convenience function to quickly define all arithmetic operations:

In [ ]:
class tint(tint):
    def create(self, n):
        # print("New tint from", n)
        return tint(n, taint=self.taint)

The `make_int_wrapper()` function creates a wrapper around an existing `int` method which attaches the taint to the result of the method:

In [ ]:
def make_int_wrapper(fun):
    def proxy(self, *args, **kwargs):
        res = fun(self, *args, **kwargs)
        # print(fun, args, kwargs, "=", repr(res))
        return self.create(res)
    return proxy

We do this for all arithmetic operators:

In [ ]:
for name in ['__add__', '__radd__', '__mul__', '__rmul__', '__sub__', 
             '__floordiv__', '__truediv__']:
    fun = getattr(int, name)
    setattr(tint, name, make_int_wrapper(fun))

In [ ]:
x = tint(42, taint='SECRET')
y = x + 1
y.taint

#### Part 3: Passing taints from integers to strings

Converting a tainted integer into a string (using `repr()`) should yield a tainted string:
```python
s = repr(x)
assert s.taint == 'SECRET'
```

**Solution.** We define the string conversion functions such that they return a tainted string (`tstr`):

In [ ]:
class tint(tint):
    def __repr__(self):
        s = int.__repr__(self)
        return tstr(s, taint=self.taint)

In [ ]:
class tint(tint):
    def __str__(self):
        return tstr(int.__str__(self), taint=self.taint)

In [ ]:
x = tint(42, taint='SECRET')
s = repr(x)
assert s.taint == 'SECRET'

#### Part 4: Passing taints from integers to strings

Converting a tainted object (with a `taint` attribute) to an integer should pass that taint:

```python
password = tstr('1234', taint='NOT_EXACTLY_SECRET')
x = tint(password)
assert x == 1234
assert x.taint == 'NOT_EXACTLY_SECRET'
```

**Solution.** This can be done by having the `__init__()` constructor check for a `taint` attibute:

In [ ]:
class tint(tint):
    def __init__(self, value, taint=None, **kwargs):
        if taint is not None:
            self.taint = taint
        else:
            self.taint = getattr(value, 'taint', None)

In [ ]:
password = tstr('1234', taint='NOT_EXACTLY_SECRET')
x = tint(password)

In [ ]:
assert x == 1234

In [ ]:
assert x.taint == 'NOT_EXACTLY_SECRET'

### Exercise 2: Information Flow Testing

Generate tests that ensure a _maximum_ of information flow, propagating specific taints as much as possible.  Implement an appropriate fitness function for [search-based testing](SearchBasedFuzzer.ipynb) and let the search-based fuzzer search for solutions.

**Solution.** This will become a section on its own; as of now, it is an exercise for the reader.